In [1]:
import pandas as pd
import pandas.tseries.offsets as offsets
import os
from sklearn.preprocessing import OneHotEncoder
import pandas_profiling as pdp
from sklearn.preprocessing import LabelEncoder
import japandas as jpd
import jpholiday
import datetime
from IPython.display import display
import numpy as np


In [2]:
pd.set_option("display.max_columns", 200)
pd.set_option("display.max_rows", 400)

In [3]:
train = pd.read_csv("../data/train.csv")
pdp.ProfileReport(train)


Number of variables,13
Number of observations,3366
Total Missing (%),0.0%
Total size in memory,341.9 KiB
Average record size in memory,104.0 B
Numeric,4
Categorical,9
Boolean,0
Date,0
Text (Unique),0
Rejected,0


In [452]:
#train.drop(2479, inplace=True)  # 無観客試合 id=15699


In [4]:
ex_train = pd.read_csv("../data/ex_total.csv")
ex_train["match_date"] = pd.to_datetime(ex_train["match_date"])
ex_train = ex_train.loc[ex_train["match_date"] < pd.to_datetime("2006-3-3"),:]
print(ex_train.shape)
pdp.ProfileReport(ex_train)


(5092, 14)


Number of variables,15
Number of observations,5092
Total Missing (%),2.8%
Total size in memory,596.8 KiB
Average record size in memory,120.0 B
Numeric,4
Categorical,8
Boolean,0
Date,1
Text (Unique),1
Rejected,1


In [5]:
#J1のデータだけ使う
ex_train = ex_train.loc[ex_train["division"] == 1,:]
print(ex_train.shape)

(3372, 14)


In [6]:
test = pd.read_csv("../data/test.csv")
pdp.ProfileReport(test)


Number of variables,12
Number of observations,477
Total Missing (%),0.9%
Total size in memory,44.8 KiB
Average record size in memory,96.2 B
Numeric,3
Categorical,9
Boolean,0
Date,0
Text (Unique),0
Rejected,0


In [7]:
#total = pd.concat((ex_train,train))
#total = pd.concat((total,test))
total = pd.concat((train,test))
total = total.reset_index(drop=True)
print(total.shape)
total.head()

(3843, 13)


,attendance,away_team,broadcasters,home_team,humidity,id,kick_off_time,match_date,round,section,temperature,venue,weather
0,20916.0,浦和,NHK総合/J SPORTS(録),G大阪,40.0,9190,16:04,2006-03-04,第1日,第1節,8.3,万博記念競技場,晴
1,14277.0,清水,山梨放送/テレビ静岡(録)/J SPORTS(録),甲府,28.0,9191,13:00,2006-03-05,第2日,第1節,12.9,山梨県小瀬スポーツ公園陸上競技場,晴
2,22531.0,大分,BS-i/MXテレビ(録)/J SPORTS(録),FC東京,35.0,9192,13:35,2006-03-05,第2日,第1節,12.1,味の素スタジアム,晴
3,28564.0,福岡,J SPORTS,磐田,42.0,9193,14:04,2006-03-05,第2日,第1節,11.6,静岡スタジアムエコパ,晴
4,17199.0,C大阪,スカイパーフェクTV!/NHK名古屋(録)/NHK大阪(録)/J SPORTS(録),名古屋,32.0,9194,14:04,2006-03-05,第2日,第1節,13.1,名古屋市瑞穂陸上競技場,晴


In [8]:
#表記ゆれ統一
total["away_team"] = total["away_team"].str.h2z()
total["home_team"] = total["home_team"].str.h2z()
total["venue"] = total["venue"].str.h2z()
pdp.ProfileReport(total)


Number of variables,13
Number of observations,3843
Total Missing (%),1.1%
Total size in memory,390.4 KiB
Average record size in memory,104.0 B
Numeric,4
Categorical,9
Boolean,0
Date,0
Text (Unique),0
Rejected,0


In [9]:
match_report = pd.read_csv("../data/match_reports.csv")
print(match_report.shape)
match_report.tail()

(3825, 25)


,id,home_team_player11,home_team_player10,home_team_player9,home_team_player8,home_team_player7,home_team_player6,home_team_player5,home_team_player4,home_team_player3,home_team_player2,home_team_player1,home_team_score,away_team_score,away_team_player1,away_team_player2,away_team_player3,away_team_player4,away_team_player5,away_team_player6,away_team_player7,away_team_player8,away_team_player9,away_team_player10,away_team_player11
3820,20893,8 髙萩 洋次郎 FW,9 ディエゴ オリヴェイラ FW,27 田邉 草民 MF,18 橋本 拳人 MF,7 米本 拓司 MF,38 東 慶悟 MF,6 太田 宏介 DF,3 森重 真人 DF,29 岡崎 慎 DF,2 室屋 成 DF,1 大久保 択生 GK,5,2,21 飯倉 大樹 GK,27 松原 健 DF,22 中澤 佑二 DF,13 金井 貢史 DF,24 山中 亮輔 DF,6 扇原 貴宏 MF,5 喜田 拓也 MF,14 天野 純 MF,19 仲川 輝人 FW,16 伊藤 翔 FW,11 遠藤 渓太 FW
3821,20894,11 小林 悠 FW,8 阿部 浩之 MF,41 家長 昭博 MF,14 中村 憲剛 MF,10 大島 僚太 MF,25 守田 英正 MF,7 車屋 紳太郎 DF,5 谷口 彰悟 DF,3 奈良 竜樹 DF,18 エウシーニョ DF,1 チョン ソンリョン GK,1,0,30 徳重 健太 GK,32 徳永 悠平 DF,4 髙杉 亮太 DF,5 田上 大地 DF,10 黒木 聖仁 MF,15 島田 譲 MF,3 飯尾 竜太朗 MF,28 翁長 聖 MF,19 澤田 崇 MF,11 鈴木 武蔵 FW,9 ファンマ FW
3822,20895,9 アデミウソン FW,11 ファン ウィジョ FW,10 倉田 秋 MF,29 髙江 麗央 MF,8 マテウス MF,7 遠藤 保仁 MF,6 初瀬 亮 DF,13 菅沼 駿哉 DF,5 三浦 弦太 DF,14 米倉 恒貴 DF,1 東口 順昭 GK,1,2,13 六反 勇治 GK,28 立田 悠悟 DF,3 ファン ソッコ DF,4 フレイレ DF,25 松原 后 DF,10 白崎 凌兵 MF,17 河井 陽介 MF,30 金子 翔太 MF,29 石毛 秀樹 MF,20 クリスラン FW,23 北川 航也 FW
3823,20896,13 高木 俊幸 FW,9 杉本 健勇 FW,24 山村 和也 MF,17 福満 隆貴 MF,11 ソウザ MF,6 山口 蛍 MF,22 マテイ ヨニッチ DF,15 木本 恭生 DF,14 丸橋 祐介 DF,2 松田 陸 DF,21 キム ジンヒョン GK,1,1,1 西川 周作 GK,6 遠藤 航 DF,2 マウリシオ DF,5 槙野 智章 DF,27 橋岡 大樹 MF,16 青木 拓矢 MF,10 柏木 陽介 MF,3 宇賀神 友弥 MF,9 武藤 雄樹 MF,12 ファブリシオ MF,30 興梠 慎三 FW
3824,20897,27 田川 亨介 FW,22 池田 圭 FW,40 小野 裕二 FW,4 原川 力 MF,6 福田 晃斗 MF,36 高橋 秀人 MF,23 吉田 豊 DF,3 高橋 祐治 DF,5 キム ミンヒョク DF,8 藤田 優人 DF,20 権田 修一 GK,0,1,21 関 憲太郎 GK,13 平岡 康裕 DF,27 大岩 一貴 DF,6 板倉 滉 DF,17 富田 晋伍 MF,15 矢島 慎也 MF,4 蜂須賀 孝治 MF,23 中野 嘉大 MF,7 奥埜 博亮 FW,20 阿部 拓馬 FW,11 石原 直樹 FW


In [10]:
stadium = pd.read_csv("../data/stadium_capacity_mapping.csv")
print(stadium.shape)
stadium.head()

(99, 2)


,stadium,capacity
0,日産スタジアム,72081
1,埼玉スタジアム2002,62010
2,国立競技場,54224
3,静岡スタジアムエコパ,51697
4,エコパスタジアム,51697


In [11]:
ex_stadium = pd.read_csv("../data/ex_stadium_capacity_mapping_padding.csv")
print(ex_stadium.shape)
ex_stadium

(166, 2)


,stadium,capacity
0,国立霞ヶ丘競技場,48000.0
1,横浜市三ツ沢総合公園球技場,15454.0
2,広島スタジアム,33000.0
3,県立カシマサッカースタジアム,40728.0
4,万博記念競技場,21000.0
5,日本平運動公園球技場,20339.0
6,浦和市駒場スタジアム,21500.0
7,名古屋市瑞穂球技場,15000.0
8,広島ビッグアーチ,35000.0
9,静岡県営草薙陸上競技場,28000.0


In [12]:
s1 = set(stadium["stadium"])
s2 = set(ex_stadium["stadium"])
dupe = s1.intersection(s2)
dupe

{'えがお健康スタジアム',
 'さいたま市浦和駒場スタジアム',
 'とりぎんバードスタジアム',
 'アウトソーシングスタジアム日本平',
 'エコパスタジアム',
 'エディオンスタジアム広島',
 'キンチョウスタジアム',
 'ケーズデンキスタジアム水戸',
 'コカ・コーラウエスト広島スタジアム',
 'シティライトスタジアム',
 'デンカビッグスワンスタジアム',
 'トランスコスモススタジアム長崎',
 'ニッパツ三ツ沢球技場',
 'ニンジニアスタジアム',
 'ノエビアスタジアム神戸',
 'パナソニックスタジアム吹田',
 'パロマ瑞穂スタジアム',
 'フクダ電子アリーナ',
 'ベストアメニティスタジアム',
 'ホームズスタジアム神戸',
 'ヤンマースタジアム長居',
 'ユアテックスタジアム仙台',
 'レベルファイブスタジアム',
 '万博記念競技場',
 '三協フロンテア柏スタジアム',
 '下関市営下関陸上競技場',
 '九州石油ドーム',
 '京都市西京極総合運動公園陸上競技場兼球技場',
 '佐賀県総合運動場陸上競技場',
 '名古屋市瑞穂球技場',
 '名古屋市瑞穂陸上競技場',
 '味の素スタジアム',
 '味の素フィールド西が丘',
 '国立競技場',
 '大分銀行ドーム',
 '大阪長居スタジアム',
 '宮城スタジアム',
 '富山県総合運動公園陸上競技場',
 '山梨中銀スタジアム',
 '山梨県小瀬スポーツ公園陸上競技場',
 '岐阜メモリアルセンター長良川競技場',
 '岡山県津山陸上競技場',
 '市立吹田サッカースタジアム',
 '平塚競技場',
 '広島ビッグアーチ',
 '新潟スタジアム',
 '日本平スタジアム',
 '日産スタジアム',
 '日立柏サッカー場',
 '札幌ドーム',
 '札幌厚別公園競技場',
 '東北電力ビッグスワンスタジアム',
 '東平尾公園博多の森球技場',
 '松本平広域公園総合球技場',
 '柏の葉公園総合競技場',
 '栃木県グリーンスタジアム',
 '横浜市三ツ沢公園球技場',
 '横浜市三ツ沢公園陸上競技場',
 '正田醤油スタジアム群馬',
 '江東区夢の島競技場',
 '熊本市水前寺競技場',
 '熊本県民総合運動公園陸上競技場',
 '熊谷スポーツ文化公園

In [13]:
dupe_index = []
for key,row in  ex_stadium.iterrows():
    if dupe.issuperset(set([row["stadium"]])):
        dupe_index.append(key)

In [14]:
ex_stadium = ex_stadium.drop(dupe_index)

In [15]:
total_stadium = pd.concat((stadium, ex_stadium))
total_stadium = total_stadium.reset_index(drop=True)
total_stadium

,stadium,capacity
0,日産スタジアム,72081.0
1,埼玉スタジアム2002,62010.0
2,国立競技場,54224.0
3,静岡スタジアムエコパ,51697.0
4,エコパスタジアム,51697.0
5,宮城スタジアム,49133.0
6,ひとめぼれスタジアム宮城,49133.0
7,味の素スタジアム,48999.0
8,ヤンマースタジアム長居,47853.0
9,大阪長居スタジアム,47853.0


In [16]:
match = pd.read_csv("../data/match_reports.csv")
print(match.shape)
match.head()

(3825, 25)


,id,home_team_player11,home_team_player10,home_team_player9,home_team_player8,home_team_player7,home_team_player6,home_team_player5,home_team_player4,home_team_player3,home_team_player2,home_team_player1,home_team_score,away_team_score,away_team_player1,away_team_player2,away_team_player3,away_team_player4,away_team_player5,away_team_player6,away_team_player7,away_team_player8,away_team_player9,away_team_player10,away_team_player11
0,9190,9 マグノ アウベス FW,8 フェルナンジーニョ FW,10 二川 孝広 MF,7 遠藤 保仁 MF,27 橋本 英郎 MF,17 明神 智和 MF,14 家長 昭博 DF,6 山口 智 DF,2 シジクレイ DF,21 加地 亮 DF,22 藤ヶ谷 陽介 GK,1,1,23 都築 龍太 GK,2 坪井 慶介 DF,4 田中 マルクス闘莉王 DF,20 堀之内 聖 DF,6 山田 暢久 MF,17 長谷部 誠 MF,13 鈴木 啓太 MF,8 三都主 アレサンドロ MF,18 小野 伸二 MF,10 ポンテ FW,21 ワシントン FW
1,9191,18 長谷川 太郎 FW,16 バレー FW,11 宇留野 純 FW,31 林 健太郎 MF,25 鈴木 健太 MF,8 倉貫 一毅 MF,4 山本 英臣 DF,20 ビジュ DF,15 アライール DF,32 杉山 新 DF,1 阿部 謙作 GK,0,2,21 西部 洋平 GK,25 市川 大祐 DF,26 青山 直晃 DF,4 高木 和道 DF,3 山西 尊裕 DF,13 兵働 昭弘 MF,7 伊東 輝悦 MF,16 枝村 匠馬 MF,10 藤本 淳吾 MF,18 チョ ジェジン FW,17 マルキーニョス FW
2,9192,9 ルーカス FW,21 ササ サルセード FW,35 リチェーリ FW,6 今野 泰幸 MF,23 梶山 陽平 MF,19 伊野波 雅彦 MF,15 鈴木 規郎 DF,5 増嶋 竜也 DF,2 茂庭 照幸 DF,25 徳永 悠平 DF,1 土肥 洋一 GK,2,0,1 西川 周作 GK,2 三木 隆司 DF,4 深谷 友基 DF,22 上本 大海 DF,5 エジミウソン MF,11 トゥーリオ MF,6 梅田 高志 MF,17 根本 裕一 MF,26 内村 圭宏 MF,13 高松 大樹 FW,9 オズマール FW
3,9193,15 西野 泰正 FW,22 カレン ロバート FW,7 名波 浩 MF,14 村井 慎二 MF,23 福西 崇史 MF,25 ファブリシオ MF,11 西 紀寛 MF,6 服部 年宏 DF,5 田中 誠 DF,2 鈴木 秀人 DF,1 川口 能活 GK,1,1,1 水谷 雄一 GK,22 中村 北斗 DF,4 金古 聖司 DF,5 千代反田 充 DF,3 アレックス DF,7 宮崎 光平 MF,6 布部 陽功 MF,8 ホベルト MF,14 古賀 誠史 MF,27 田中 佑昌 FW,36 グラウシオ FW
4,9194,19 杉本 恵太 FW,11 玉田 圭司 FW,27 片山 奨典 MF,7 中村 直志 MF,14 吉村 圭司 MF,8 金 正友 MF,6 有村 光史 DF,29 深津 康太 DF,5 古賀 正紘 DF,4 大森 征之 DF,22 川島 永嗣 GK,3,2,22 吉田 宗弘 GK,2 ブルーノ クアドロス DF,5 前田 和哉 DF,3 柳本 啓成 DF,4 河村 崇大 MF,23 下村 東美 MF,10 ピンゴ MF,6 ゼ カルロス MF,8 森島 寛晃 FW,20 西澤 明訓 FW,9 古橋 達弥 FW


In [17]:
holiday = pd.read_csv("../data/holidays_in_japan.csv")
print(holiday.shape)
holiday.head()

(242, 2)


,holiday_date,description
0,2006-01-01,New Years Day
1,2006-01-02,New Years Holiday
2,2006-01-03,New Years Holiday
3,2006-01-09,Coming-of-age Day
4,2006-02-11,National Foundation Day


# team labelencoder

In [18]:
team_df  = total.loc[:,["home_team","away_team"]] 

In [19]:
le = LabelEncoder()
hometeam_label = le.fit_transform(team_df["home_team"])
hometeam_label

array([29, 20, 28, ...,  3, 22, 24])

In [20]:
hometeam_label_df = pd.DataFrame(hometeam_label)
hometeam_label_df.columns = ["hometeam_label"]

In [21]:
awayteam_label = le.transform(team_df["away_team"])
awayteam_label

array([17, 18,  4, ..., 19,  1, 18])

In [22]:
awayteam_label_df = pd.DataFrame(awayteam_label)
awayteam_label_df.columns = ["awayteam_label"]

In [23]:
feature = total[["id","match_date","attendance"]]
feature = pd.concat((feature,hometeam_label_df),axis = 1)
feature = pd.concat((feature,awayteam_label_df),axis = 1)
print(feature.shape)
feature.columns = ["id","date","attendance","hometeam_label","awayteam_label"]

feature.to_csv("../data/team.csv", index=False)

(3843, 5)


In [24]:
pd.DataFrame(le.classes_).to_csv("../data/team_label.csv", index=False)

In [25]:
pd.DataFrame(le.classes_)

,0
0,京都
1,仙台
2,千葉
3,名古屋
4,大分
5,大宮
6,山形
7,川崎Ｆ
8,広島
9,徳島


# 休日ラベル

In [26]:
date = pd.date_range(start='1993-1-1', end='2018-12-31', freq='D')
date = pd.Series(date)


In [27]:
public_holiday = []
for i, d in date.iteritems():
    public_holiday.append(jpholiday.is_holiday(datetime.date(d.to_datetime().year,d.to_datetime().month,d.to_datetime().day)))

date_df = pd.DataFrame(date)
date_df.columns = ["date"]
p_holiday = pd.DataFrame(public_holiday)
p_holiday.columns = ["public_holiday"]

p_holiday_df = pd.concat((date_df,p_holiday),axis=1)
p_holiday_df["public_holiday"] = p_holiday_df["public_holiday"].astype(int)

/home/watanabe/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2881: FutureWarning: to_datetime is deprecated. Use self.to_pydatetime()
  exec(code_obj, self.user_global_ns, self.user_ns)


In [28]:
p_holiday_df

,date,public_holiday
0,1993-01-01,1
1,1993-01-02,0
2,1993-01-03,0
3,1993-01-04,0
4,1993-01-05,0
5,1993-01-06,0
6,1993-01-07,0
7,1993-01-08,0
8,1993-01-09,0
9,1993-01-10,0


In [29]:
weekday = []
for d in date:
    weekday.append(d.weekday())
    
weekday_df = pd.DataFrame(weekday)
weekday_df.columns = ["weekday"]
data = pd.concat((p_holiday_df,weekday_df),axis=1)

In [30]:
data.loc[(data["weekday"]==5),"holiday"] = True
data.loc[(data["weekday"]==6),"holiday"] = True

data = data.fillna(False)
data["holiday"] = data["holiday"].astype(int)
data["or_holiday"] = (data["public_holiday"]) | (data["holiday"])
print(data.shape)
data.head()

(9496, 5)


,date,public_holiday,weekday,holiday,or_holiday
0,1993-01-01,1,4,0,1
1,1993-01-02,0,5,1,1
2,1993-01-03,0,6,1,1
3,1993-01-04,0,0,0,0
4,1993-01-05,0,1,0,0


In [31]:
data["or_holiday"].isnull().value_counts()

False    9496
Name: or_holiday, dtype: int64

In [32]:
data.to_csv("../data/holiday.csv", index=False)

# キックオフ時刻encoder

In [33]:
kickoff = total.loc[:,["id","kick_off_time"]]
kickoff["kick_off_time"] = pd.to_datetime(kickoff["kick_off_time"])
print(kickoff.shape)
kickoff.head()

(3843, 2)


,id,kick_off_time
0,9190,2018-11-08 16:04:00
1,9191,2018-11-08 13:00:00
2,9192,2018-11-08 13:35:00
3,9193,2018-11-08 14:04:00
4,9194,2018-11-08 14:04:00


In [34]:
kickoff["kick_off_time_hour"] = kickoff["kick_off_time"].apply(lambda x: x.round("H"))
kickoff["kick_off_time_hour"] = kickoff["kick_off_time_hour"].apply(lambda x: x.hour)

In [35]:
kickoff.head()

,id,kick_off_time,kick_off_time_hour
0,9190,2018-11-08 16:04:00,16
1,9191,2018-11-08 13:00:00,13
2,9192,2018-11-08 13:35:00,14
3,9193,2018-11-08 14:04:00,14
4,9194,2018-11-08 14:04:00,14


In [36]:
le = LabelEncoder()
kickoff_label = le.fit_transform(kickoff["kick_off_time_hour"])
kickoff_label

array([3, 0, 1, ..., 1, 1, 1])

In [37]:
kickoff["kickoff_hour_label"] =  kickoff_label
kickoff.head()

,id,kick_off_time,kick_off_time_hour,kickoff_hour_label
0,9190,2018-11-08 16:04:00,16,3
1,9191,2018-11-08 13:00:00,13,0
2,9192,2018-11-08 13:35:00,14,1
3,9193,2018-11-08 14:04:00,14,1
4,9194,2018-11-08 14:04:00,14,1


In [38]:
kickoff.to_csv("../data/kickoff.csv",index=False)

In [39]:
pd.DataFrame(le.classes_).to_csv("../data/kickoff_label.csv", index=False)
le.classes_

array([13, 14, 15, 16, 17, 18, 19, 20])

# スタジアム

In [40]:
stadiam = total[["id", "venue"]]
stadiam.head()

,id,venue
0,9190,万博記念競技場
1,9191,山梨県小瀬スポーツ公園陸上競技場
2,9192,味の素スタジアム
3,9193,静岡スタジアムエコパ
4,9194,名古屋市瑞穂陸上競技場


In [41]:
stadium_data = pd.merge(stadiam,total_stadium,left_on="venue",right_on="stadium",how="left")
stadium_data

,id,venue,stadium,capacity
0,9190,万博記念競技場,万博記念競技場,21000.0
1,9191,山梨県小瀬スポーツ公園陸上競技場,山梨県小瀬スポーツ公園陸上競技場,15859.0
2,9192,味の素スタジアム,味の素スタジアム,48999.0
3,9193,静岡スタジアムエコパ,静岡スタジアムエコパ,51697.0
4,9194,名古屋市瑞穂陸上競技場,名古屋市瑞穂陸上競技場,20223.0
5,9195,埼玉スタジアム２００２,埼玉スタジアム２００２,63700.0
6,9196,等々力陸上競技場,等々力陸上競技場,26827.0
7,9197,広島ビッグアーチ,広島ビッグアーチ,36894.0
8,9198,日産スタジアム,日産スタジアム,72081.0
9,9199,埼玉スタジアム２００２,埼玉スタジアム２００２,63700.0


In [42]:
stadium_data = pd.merge(stadiam,total_stadium,left_on="venue",right_on="stadium",how="left")
stadium_data["venue"] = stadium_data["venue"].astype(str)
le = LabelEncoder()
stadium_label = le.fit_transform(stadium_data["venue"])
stadium_label

array([19, 34, 26, ..., 11,  9,  7])

In [43]:
stadium_data["stadium_label"] = stadium_label
stadium_data.head()

,id,venue,stadium,capacity,stadium_label
0,9190,万博記念競技場,万博記念競技場,21000.0,19
1,9191,山梨県小瀬スポーツ公園陸上競技場,山梨県小瀬スポーツ公園陸上競技場,15859.0,34
2,9192,味の素スタジアム,味の素スタジアム,48999.0,26
3,9193,静岡スタジアムエコパ,静岡スタジアムエコパ,51697.0,56
4,9194,名古屋市瑞穂陸上競技場,名古屋市瑞穂陸上競技場,20223.0,25


In [44]:
stadium_data.to_csv("../data/stadiam.csv", index=False)

In [45]:
pd.DataFrame(le.classes_).to_csv("../data/stadiam_label.csv", index=False)
le.classes_

array(['さいたま市浦和駒場スタジアム', 'アウトソーシングスタジアム日本平', 'エコパスタジアム', 'エディオンスタジアム広島',
       'キンチョウスタジアム', 'コカ・コーラウエスト広島スタジアム', 'デンカビッグスワンスタジアム',
       'トランスコスモススタジアム長崎', 'ニッパツ三ツ沢球技場', 'ノエビアスタジアム神戸', 'パナソニックスタジアム吹田',
       'パロマ瑞穂スタジアム', 'フクダ電子アリーナ', 'ベストアメニティスタジアム', 'ホームズスタジアム神戸',
       'ヤマハスタジアム（磐田）', 'ヤンマースタジアム長居', 'ユアテックスタジアム仙台', 'レベルファイブスタジアム',
       '万博記念競技場', '三協フロンテア柏スタジアム', '九州石油ドーム', '京都市西京極総合運動公園陸上競技場兼球技場',
       '佐賀県総合運動場陸上競技場', '名古屋市瑞穂球技場', '名古屋市瑞穂陸上競技場', '味の素スタジアム', '国立競技場',
       '埼玉スタジアム２００２', '大分銀行ドーム', '大阪長居スタジアム', '大阪長居第２陸上競技場', '宮城スタジアム',
       '山梨中銀スタジアム', '山梨県小瀬スポーツ公園陸上競技場', '市立吹田サッカースタジアム', '平塚競技場',
       '広島ビッグアーチ', '新潟スタジアム', '日本平スタジアム', '日産スタジアム', '日立柏サッカー場', '札幌ドーム',
       '札幌厚別公園競技場', '東北電力ビッグスワンスタジアム', '東平尾公園博多の森球技場', '松本平広域公園総合球技場',
       '柏の葉公園総合競技場', '横浜市三ツ沢公園球技場', '熊本県民総合運動公園陸上競技場', '熊谷スポーツ文化公園陸上競技場',
       '県立カシマサッカースタジアム', '石川県西部緑地公園陸上競技場', '神戸総合運動公園ユニバー記念競技場',
       '等々力陸上競技場', '豊田スタジアム', '静岡スタジアムエコパ', '鳴門・大塚スポーツパークポカリスエットスタジアム',
       '鹿児島県立鴨池陸上競技場',

# month

In [46]:
total["match_date"] = pd.to_datetime(total["match_date"])

In [47]:
total["match_date_month"] = total["match_date"].dt.month

In [48]:
month = total.loc[:,["id","match_date_month"]]

In [49]:
month.head()

,id,match_date_month
0,9190,3
1,9191,3
2,9192,3
3,9193,3
4,9194,3


In [50]:
month.to_csv("../data/month.csv", index=False)

# weather

In [51]:
weather = pd.DataFrame()
weather["id"] = total["id"]
#weather["weather"] = total["weather"].str[:1] #先頭の文字を天気とする
#weather["weather"] = total["weather"].fillna("晴") #testデータの33,34期は天気がnan、対処必要 とりあえず晴でうめる
display(weather)

,id
0,9190
1,9191
2,9192
3,9193
4,9194
5,9195
6,9196
7,9197
8,9198
9,9199


In [52]:
weather = pd.concat((weather,total["weather"].str[:1]),axis=1)
weather["weather"] = weather["weather"].fillna("晴")

In [53]:
weather.head()

,id,weather
0,9190,晴
1,9191,晴
2,9192,晴
3,9193,晴
4,9194,晴


In [54]:
le = LabelEncoder()
weather_label = le.fit_transform(weather["weather"].astype(str))
weather_label

array([1, 1, 1, ..., 1, 1, 1])

In [55]:
weather["weather_label"] = weather_label

In [56]:
pd.DataFrame(le.classes_).to_csv("../data/weather_label.csv", index=False)
le.classes_

array(['屋', '晴', '曇', '雨', '雪', '雷', '霧'], dtype=object)

In [57]:
weather.to_csv("../data/weather.csv", index=False)

# 順位

勝点計算

In [58]:
#勝、引き分け、負けフラグ
total_match = pd.merge(total,match_report,on="id")
total_match["match_date"] = pd.to_datetime(total_match["match_date"])

In [59]:
total_match["home_win"] = total_match["home_team_score"] > total_match["away_team_score"]
total_match["away_win"] = total_match["home_team_score"] < total_match["away_team_score"]
total_match["draw"] = total_match["home_team_score"] == total_match["away_team_score"]

In [60]:
target_years = list(range(2006,2018))
target_years

[2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017]

In [67]:
for year in target_years:
    print(year)
    match = total_match.loc[(total_match["match_date"] >= pd.to_datetime(str(year)+"-2-15"))&(total_match["match_date"] <pd.to_datetime(str(year+1)+"-2-15")),:]
    attend_teams = set(match["home_team"]) & set(match["away_team"])
    match.loc[match["home_win"],"winner"] = match["home_team"]
    match.loc[match["away_win"],"winner"] = match["away_team"]
    score_dic = {}

    for team in attend_teams:
        tmp = match.loc[(match["home_team"] == team)|(match["away_team"] == team),:]
        tmp=tmp.reset_index(drop=True)
    
        score  = []
        for key,row in tmp.iterrows():
            if row["winner"] == team:
                score.append(3)
            elif row["draw"] == True:
                score.append(1)
            else:
                score.append(0)
            
        score_dic[team] = np.cumsum(score)
    

    df = pd.DataFrame(score_dic)
    df.to_csv("../data/vpoint"+str(year)+".csv", index=False)
    display(df.head())

2006


/home/watanabe/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:297: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/home/watanabe/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:477: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


,京都,千葉,名古屋,大分,大宮,川崎Ｆ,広島,新潟,横浜ＦＭ,浦和,清水,甲府,磐田,福岡,鹿島,Ｃ大阪,ＦＣ東京,Ｇ大阪
0,0,0,3,0,3,3,0,0,3,1,3,0,1,1,3,0,3,1
1,0,1,3,1,4,6,1,3,6,4,6,1,1,2,3,0,3,4
2,1,2,4,4,4,6,1,6,9,7,9,4,2,3,4,0,3,4
3,2,5,7,4,4,7,2,6,12,10,9,4,5,3,7,0,4,7
4,2,5,7,7,7,10,2,6,12,13,9,5,5,4,10,3,7,10


2007


,千葉,名古屋,大分,大宮,川崎Ｆ,広島,新潟,柏,横浜ＦＣ,横浜ＦＭ,浦和,清水,甲府,磐田,神戸,鹿島,ＦＣ東京,Ｇ大阪
0,0,3,1,0,3,3,1,3,0,3,3,3,0,0,0,0,0,3
1,0,6,1,0,4,4,2,4,3,3,4,6,0,3,1,0,3,6
2,1,9,4,0,7,4,2,7,3,3,7,6,0,6,4,1,3,9
3,1,12,5,0,7,4,5,10,3,6,8,9,0,9,5,2,3,12
4,4,12,5,1,8,7,8,13,3,6,11,12,3,9,5,3,3,13


...

,仙台,名古屋,山形,川崎Ｆ,広島,新潟,松本,柏,横浜ＦＭ,浦和,清水,湘南,甲府,神戸,鳥栖,鹿島,ＦＣ東京,Ｇ大阪
0,3,1,0,3,3,0,1,3,0,3,3,0,0,0,3,0,1,1
1,4,1,0,4,6,1,1,4,1,6,4,3,3,1,6,0,2,1
2,5,2,3,4,7,4,4,4,4,7,4,4,3,1,6,1,5,4
3,8,2,3,7,7,4,4,4,7,10,4,7,3,4,6,4,8,7
4,9,5,3,8,7,5,4,7,8,11,4,7,3,7,9,5,11,10


2016


,仙台,名古屋,大宮,川崎Ｆ,広島,新潟,柏,横浜ＦＭ,浦和,湘南,甲府,磐田,神戸,福岡,鳥栖,鹿島,ＦＣ東京,Ｇ大阪
0,3,3,3,3,0,3,0,0,3,0,3,0,0,0,3,3,0,0
1,3,4,6,4,1,3,0,1,3,1,3,3,3,1,3,6,3,3
2,6,4,6,7,2,3,1,4,6,2,4,4,3,1,4,6,6,6
3,6,7,6,10,5,4,2,7,9,2,4,5,6,2,4,9,6,6
4,6,7,7,11,8,7,3,10,12,2,4,6,9,2,5,10,9,6


2017


,仙台,大宮,川崎Ｆ,広島,新潟,札幌,柏,横浜ＦＭ,浦和,清水,甲府,磐田,神戸,鳥栖,鹿島,Ｃ大阪,ＦＣ東京,Ｇ大阪
0,3,0,3,1,1,0,3,3,0,0,1,1,3,0,0,1,3,1
1,6,0,4,1,1,0,3,6,3,3,1,1,6,1,3,1,6,4
2,6,0,7,1,1,1,3,6,6,6,1,4,9,4,6,2,6,7
3,9,0,7,1,2,4,3,7,7,6,4,4,12,4,9,5,9,8
4,9,0,10,1,2,4,6,7,10,6,7,7,12,5,12,8,10,11


In [83]:
from scipy import stats

for year in target_years:
    vp = pd.read_csv("../data/vpoint"+str(year)+".csv")
    order = vp
    for key,row in vp.iterrows():
        order.loc[key,:] = stats.mstats.rankdata(row)
    order["section"]  = order.index.values + 1
    order["year"]  = year
    tmp = pd.melt(order ,id_vars=['section',"year"], var_name='team', value_name='order')
    display(tmp)
    tmp.to_csv("../data/order"+str(year)+".csv", index=False)
    

,section,year,team,order
0,1,2006,京都,4.0
1,2,2006,京都,1.5
2,3,2006,京都,2.5
3,4,2006,京都,2.5
4,5,2006,京都,1.5
5,6,2006,京都,4.0
6,7,2006,京都,4.0
7,8,2006,京都,4.0
8,9,2006,京都,3.0
9,10,2006,京都,3.0


,section,year,team,order
0,1,2007,千葉,4.5
1,2,2007,千葉,2.5
2,3,2007,千葉,3.5
3,4,2007,千葉,3.0
4,5,2007,千葉,6.0
5,6,2007,千葉,3.5
6,7,2007,千葉,5.0
7,8,2007,千葉,4.5
8,9,2007,千葉,4.5
9,10,2007,千葉,5.0


,section,year,team,order
0,1,2008,京都,9.5
1,2,2008,京都,13.5
2,3,2008,京都,12.5
3,4,2008,京都,7.5
4,5,2008,京都,12.0
5,6,2008,京都,13.0
6,7,2008,京都,11.5
7,8,2008,京都,9.0
8,9,2008,京都,7.0
9,10,2008,京都,8.5


,section,year,team,order
0,1,2009,京都,15.0
1,2,2009,京都,10.5
2,3,2009,京都,15.0
3,4,2009,京都,11.0
4,5,2009,京都,13.5
5,6,2009,京都,11.5
6,7,2009,京都,8.5
7,8,2009,京都,6.5
8,9,2009,京都,4.0
9,10,2009,京都,6.5


,section,year,team,order
0,1,2010,京都,4.0
1,2,2010,京都,4.0
2,3,2010,京都,10.0
3,4,2010,京都,12.5
4,5,2010,京都,10.0
5,6,2010,京都,8.0
6,7,2010,京都,7.5
7,8,2010,京都,4.5
8,9,2010,京都,3.5
9,10,2010,京都,2.0


...

,section,year,team,order
0,1,2013,仙台,9.5
1,2,2013,仙台,5.5
2,3,2013,仙台,10.0
3,4,2013,仙台,8.5
4,5,2013,仙台,7.0
5,6,2013,仙台,10.0
6,7,2013,仙台,7.0
7,8,2013,仙台,7.0
8,9,2013,仙台,6.5
9,10,2013,仙台,9.0


,section,year,team,order
0,1,2014,仙台,4.0
1,2,2014,仙台,2.0
2,3,2014,仙台,2.5
3,4,2014,仙台,2.0
4,5,2014,仙台,2.0
5,6,2014,仙台,2.0
6,7,2014,仙台,2.0
7,8,2014,仙台,2.0
8,9,2014,仙台,2.0
9,10,2014,仙台,2.0


,section,year,team,order
0,1,2015,仙台,15.0
1,2,2015,仙台,13.5
2,3,2015,仙台,14.5
3,4,2015,仙台,16.5
4,5,2015,仙台,14.5
5,6,2015,仙台,12.0
6,7,2015,仙台,10.0
7,8,2015,仙台,6.0
8,9,2015,仙台,5.5
9,10,2015,仙台,3.5


,section,year,team,order
0,1,2016,仙台,14.0
1,2,2016,仙台,10.0
2,3,2016,仙台,14.5
3,4,2016,仙台,11.0
4,5,2016,仙台,7.0
5,6,2016,仙台,4.5
6,7,2016,仙台,3.5
7,8,2016,仙台,4.5
8,9,2016,仙台,2.5
9,10,2016,仙台,3.0


,section,year,team,order
0,1,2017,仙台,15.5
1,2,2017,仙台,16.5
2,3,2017,仙台,12.5
3,4,2017,仙台,16.0
4,5,2017,仙台,12.0
5,6,2017,仙台,9.5
6,7,2017,仙台,7.5
7,8,2017,仙台,6.0
8,9,2017,仙台,9.0
9,10,2017,仙台,7.0


In [69]:
total.loc[:, "section"] = total.loc[:, "section"].apply(lambda x: int(x[1:-1]))
total.loc[:, "round"] = total.loc[:, "round"].apply(lambda x: int(x[1:-1]))

In [70]:
total["match_date_year"] = total["match_date"].dt.year
total["match_date_month"] = total["match_date"].dt.month
total["match_date_day"] = total["match_date"].dt.day

In [71]:
# 2015年と2016年は2シーズン制
total.query("(match_date_year in [2015, 2016])&(section == 1)").iloc[7:12, :]

,attendance,away_team,broadcasters,home_team,humidity,id,kick_off_time,match_date,round,section,temperature,venue,weather,match_date_month,match_date_year,match_date_day
2761,14446.0,浦和,スカパー!/スカパー!プレミアムサービス/NHK BS1,湘南,66.0,16810,19:04,2015-03-07,1,1,7.9,ＳｈｏｎａｎＢＭＷスタジアム平塚,雨,3,2015,7
2762,19736.0,鹿島,スカパー!/スカパー!プレミアムサービス/テレビ静岡,清水,45.0,16811,13:04,2015-03-08,2,1,19.2,ＩＡＩスタジアム日本平,晴,3,2015,8
2907,13373.0,広島,スカパー!/スカパー!プレミアムサービス,仙台,49.0,16956,18:04,2015-07-11,1,1,27.0,ユアテックスタジアム仙台,晴,7,2015,11
2908,11371.0,鳥栖,スカパー!/スカパー!プレミアムサービス,柏,73.0,16957,19:03,2015-07-11,1,1,25.3,日立柏サッカー場,晴,7,2015,11
2909,23793.0,ＦＣ東京,スカパー!/スカパー!プレミアムサービス,川崎Ｆ,71.0,16958,18:30,2015-07-11,1,1,26.1,等々力陸上競技場,晴,7,2015,11


In [72]:
sec_index = total.query("(match_date_year in [2015, 2016])&(match_date_month >= 7)").index
total.loc[sec_index, "section"] = total.loc[sec_index, "section"] + 17

In [73]:
total.head()


,attendance,away_team,broadcasters,home_team,humidity,id,kick_off_time,match_date,round,section,temperature,venue,weather,match_date_month,match_date_year,match_date_day
0,20916.0,浦和,NHK総合/J SPORTS(録),Ｇ大阪,40.0,9190,16:04,2006-03-04,1,1,8.3,万博記念競技場,晴,3,2006,4
1,14277.0,清水,山梨放送/テレビ静岡(録)/J SPORTS(録),甲府,28.0,9191,13:00,2006-03-05,2,1,12.9,山梨県小瀬スポーツ公園陸上競技場,晴,3,2006,5
2,22531.0,大分,BS-i/MXテレビ(録)/J SPORTS(録),ＦＣ東京,35.0,9192,13:35,2006-03-05,2,1,12.1,味の素スタジアム,晴,3,2006,5
3,28564.0,福岡,J SPORTS,磐田,42.0,9193,14:04,2006-03-05,2,1,11.6,静岡スタジアムエコパ,晴,3,2006,5
4,17199.0,Ｃ大阪,スカイパーフェクTV!/NHK名古屋(録)/NHK大阪(録)/J SPORTS(録),名古屋,32.0,9194,14:04,2006-03-05,2,1,13.1,名古屋市瑞穂陸上競技場,晴,3,2006,5


In [74]:
data = total[["id","humidity","temperature","round","section"]]
data.to_csv("../data/basic.csv",index=False)

In [75]:
import glob

In [89]:
concat_files = glob.glob('../data/order2*')
print(concat_files)

['../data/order2010.csv', '../data/order2009.csv', '../data/order2011.csv', '../data/order2013.csv', '../data/order2008.csv', '../data/order2006.csv', '../data/order2016.csv', '../data/order2017.csv', '../data/order2007.csv', '../data/order2014.csv', '../data/order2015.csv', '../data/order2012.csv']


In [90]:
list_concat_df =[]
for file in concat_files:
    df = pd.read_csv(file)
    print(df.shape)
    list_concat_df.append(df)
concat_df = pd.concat(list_concat_df, ignore_index=True)

(612, 4)
(612, 4)
(612, 4)
(612, 4)
(612, 4)
(612, 4)
(612, 4)
(612, 4)
(612, 4)
(612, 4)
(612, 4)
(612, 4)


In [91]:
concat_df

,section,year,team,order
0,1,2010,京都,4.0
1,2,2010,京都,4.0
2,3,2010,京都,10.0
3,4,2010,京都,12.5
4,5,2010,京都,10.0
5,6,2010,京都,8.0
6,7,2010,京都,7.5
7,8,2010,京都,4.5
8,9,2010,京都,3.5
9,10,2010,京都,2.0


In [92]:
total.head()

,attendance,away_team,broadcasters,home_team,humidity,id,kick_off_time,match_date,round,section,temperature,venue,weather,match_date_month,match_date_year,match_date_day
0,20916.0,浦和,NHK総合/J SPORTS(録),Ｇ大阪,40.0,9190,16:04,2006-03-04,1,1,8.3,万博記念競技場,晴,3,2006,4
1,14277.0,清水,山梨放送/テレビ静岡(録)/J SPORTS(録),甲府,28.0,9191,13:00,2006-03-05,2,1,12.9,山梨県小瀬スポーツ公園陸上競技場,晴,3,2006,5
2,22531.0,大分,BS-i/MXテレビ(録)/J SPORTS(録),ＦＣ東京,35.0,9192,13:35,2006-03-05,2,1,12.1,味の素スタジアム,晴,3,2006,5
3,28564.0,福岡,J SPORTS,磐田,42.0,9193,14:04,2006-03-05,2,1,11.6,静岡スタジアムエコパ,晴,3,2006,5
4,17199.0,Ｃ大阪,スカイパーフェクTV!/NHK名古屋(録)/NHK大阪(録)/J SPORTS(録),名古屋,32.0,9194,14:04,2006-03-05,2,1,13.1,名古屋市瑞穂陸上競技場,晴,3,2006,5


In [93]:
tmp = pd.merge(total,concat_df,left_on=["home_team","match_date_year","section"],right_on=["team","year","section"],how="left")
tmp = tmp.rename(columns = {"order":"home_team_order"})
tmp = pd.merge(tmp,concat_df,left_on=["away_team","match_date_year","section"],right_on=["team","year","section"],how="left")
tmp = tmp.rename(columns = {"order":"away_team_order"})
tmp.head()

,attendance,away_team,broadcasters,home_team,humidity,id,kick_off_time,match_date,round,section,temperature,venue,weather,match_date_month,match_date_year,match_date_day,year_x,team_x,home_team_order,year_y,team_y,away_team_order
0,20916.0,浦和,NHK総合/J SPORTS(録),Ｇ大阪,40.0,9190,16:04,2006-03-04,1,1,8.3,万博記念競技場,晴,3,2006,4,2006.0,Ｇ大阪,9.5,2006.0,浦和,9.5
1,14277.0,清水,山梨放送/テレビ静岡(録)/J SPORTS(録),甲府,28.0,9191,13:00,2006-03-05,2,1,12.9,山梨県小瀬スポーツ公園陸上競技場,晴,3,2006,5,2006.0,甲府,4.0,2006.0,清水,15.0
2,22531.0,大分,BS-i/MXテレビ(録)/J SPORTS(録),ＦＣ東京,35.0,9192,13:35,2006-03-05,2,1,12.1,味の素スタジアム,晴,3,2006,5,2006.0,ＦＣ東京,15.0,2006.0,大分,4.0
3,28564.0,福岡,J SPORTS,磐田,42.0,9193,14:04,2006-03-05,2,1,11.6,静岡スタジアムエコパ,晴,3,2006,5,2006.0,磐田,9.5,2006.0,福岡,9.5
4,17199.0,Ｃ大阪,スカイパーフェクTV!/NHK名古屋(録)/NHK大阪(録)/J SPORTS(録),名古屋,32.0,9194,14:04,2006-03-05,2,1,13.1,名古屋市瑞穂陸上競技場,晴,3,2006,5,2006.0,名古屋,15.0,2006.0,Ｃ大阪,4.0


In [94]:
order_df = tmp[["id","home_team_order","away_team_order"]]
print(order_df.shape)
order_df

(3843, 3)


,id,home_team_order,away_team_order
0,9190,9.5,9.5
1,9191,4.0,15.0
2,9192,15.0,4.0
3,9193,9.5,9.5
4,9194,15.0,4.0
5,9195,15.0,4.0
6,9196,15.0,4.0
7,9197,4.0,15.0
8,9198,15.0,4.0
9,9199,14.0,5.0


In [95]:
order_df.to_csv("../data/order.csv",index=False)

# チームラベル+休日フラグ+kickofftimeラベル+stadium_capa+month+weather

In [96]:
team = pd.read_csv("../data/team.csv")
holiday = pd.read_csv("../data/holiday.csv")
kickoff = pd.read_csv("../data/kickoff.csv")
stadiam = pd.read_csv("../data/stadiam.csv")
month = pd.read_csv("../data/month.csv")
weather = pd.read_csv("../data/weather.csv")
order = pd.read_csv("../data/order.csv")
basic = pd.read_csv("../data/basic.csv")

In [97]:
holiday["date"] = pd.to_datetime(holiday["date"])
team["date"] = pd.to_datetime(team["date"])


In [98]:
feature = pd.merge(team,holiday,on="date",how="left")
feature = pd.merge(feature,kickoff,on="id",how="left")
feature = pd.merge(feature,stadiam,on="id",how="left")
feature = pd.merge(feature,month,on="id",how="left")
feature = pd.merge(feature,weather,on="id",how="left")
feature = pd.merge(feature,order,on="id",how="left")
feature = pd.merge(feature,basic,on="id",how="left")


display(feature)
print(feature.shape)
feature.to_csv("../data/feature.csv", index=False)

,id,date,attendance,hometeam_label,awayteam_label,public_holiday,weekday,holiday,or_holiday,kick_off_time,kick_off_time_hour,kickoff_hour_label,venue,stadium,capacity,stadium_label,match_date_month,weather,weather_label,home_team_order,away_team_order,humidity,temperature,round,section
0,9190,2006-03-04,20916.0,29,17,0,5,1,1,2018-11-08 16:04:00,16,3,万博記念競技場,万博記念競技場,21000.0,19,3,晴,1,9.5,9.5,40.0,8.3,1,1
1,9191,2006-03-05,14277.0,20,18,0,6,1,1,2018-11-08 13:00:00,13,0,山梨県小瀬スポーツ公園陸上競技場,山梨県小瀬スポーツ公園陸上競技場,15859.0,34,3,晴,1,4.0,15.0,28.0,12.9,2,1
2,9192,2006-03-05,22531.0,28,4,0,6,1,1,2018-11-08 13:35:00,14,1,味の素スタジアム,味の素スタジアム,48999.0,26,3,晴,1,15.0,4.0,35.0,12.1,2,1
3,9193,2006-03-05,28564.0,21,23,0,6,1,1,2018-11-08 14:04:00,14,1,静岡スタジアムエコパ,静岡スタジアムエコパ,51697.0,56,3,晴,1,9.5,9.5,42.0,11.6,2,1
4,9194,2006-03-05,17199.0,3,27,0,6,1,1,2018-11-08 14:04:00,14,1,名古屋市瑞穂陸上競技場,名古屋市瑞穂陸上競技場,20223.0,25,3,晴,1,15.0,4.0,32.0,13.1,2,1
5,9195,2006-03-05,13085.0,5,2,0,6,1,1,2018-11-08 15:03:00,15,2,埼玉スタジアム２００２,埼玉スタジアム２００２,63700.0,28,3,晴,1,15.0,4.0,33.0,12.6,2,1
6,9196,2006-03-05,17444.0,7,10,0,6,1,1,2018-11-08 15:00:00,15,2,等々力陸上競技場,等々力陸上競技場,26827.0,54,3,晴,1,15.0,4.0,34.0,13.3,2,1
7,9197,2006-03-05,17564.0,8,26,0,6,1,1,2018-11-08 15:06:00,15,2,広島ビッグアーチ,広島ビッグアーチ,36894.0,37,3,曇,2,4.0,15.0,58.0,12.4,2,1
8,9198,2006-03-05,23607.0,16,0,0,6,1,1,2018-11-08 15:35:00,16,3,日産スタジアム,日産スタジアム,72081.0,40,3,晴,1,15.0,4.0,34.0,12.6,2,1
9,9199,2006-03-11,56512.0,17,21,0,5,1,1,2018-11-08 14:04:00,14,1,埼玉スタジアム２００２,埼玉スタジアム２００２,63700.0,28,3,晴,1,14.0,5.0,48.0,16.0,1,2


(3843, 25)


In [99]:
categorical = ["hometeam_label","awayteam_label","weekday","kick_off_time_hour","stadium_label","match_date_month","weather_label","round","section"]

In [ ]:
target = "hometeam_label"
